# Contextual Selection: From Principle to Mechanism

This notebook demonstrates the three-layer understanding from **DOCS/CONTEXTUAL_SELECTION_MECHANICS.md**.

## The Three Layers

```text
Layer 1: Abstract Principle
"Context actively selects what exists in it"
         ↓
Layer 2: Concrete Mechanics
τ/ρ thresholds, BSS(τ,ρ), Priority weights
         ↓
Layer 3: Structural Result
W ⊂ H×H filtered lattice, AM ≠ W
```

In [1]:
import sys
sys.path.insert(0, '/home/alexmy/SGS/SGS_lib/hllset_manifold')

from core.hrt import (
    HRTConfig, HRT,
    BasicHLLSet, ContextualSelection
)
from core.kernel import Kernel
from core.hllset import HLLSet

kernel = Kernel()
print("✓ Framework loaded")

✓ Framework loaded


## Layer 1: Abstract Principle

**Contextual Selection Principle:**
> A context C is not merely a passive container. It actively determines what elements can exist within it through selection criteria.

**Formal Statement:**

```text
S_C: H → {0, 1}
S_C(x) = 1 ⟺ x is selected by context C
```

**Key Insight:** The context itself performs the selection - not an external observer.

In [2]:
# Create a context (quantum physics)
context_tokens = ['quantum', 'entanglement', 'superposition', 'measurement', 'wavefunction']
context_hllset = kernel.absorb(context_tokens)
context_config = HRTConfig(p_bits=8, h_bits=16, tau=0.7, rho=0.3, epsilon=0.1)
context = BasicHLLSet(index=0, is_row=True, hllset=context_hllset, config=context_config)

print(f"Context: {context.name}")
print(f"Tokens: {context_tokens}")
print(f"\nAbstract Question: What can exist in this context?")
print(f"Answer: Elements that are 'quantum-like' - but how do we formalize this?")

Context: r_0
Tokens: ['quantum', 'entanglement', 'superposition', 'measurement', 'wavefunction']

Abstract Question: What can exist in this context?
Answer: Elements that are 'quantum-like' - but how do we formalize this?


## Layer 2: Concrete Mechanics (τ/ρ)

**The abstract principle is implemented through BSS thresholds:**

**Selection Rule:**

```text
S_C(x) = 1 ⟺ BSS_τ(C,x) ≥ τ AND BSS_ρ(C,x) ≤ ρ
```

where:
- **BSS_τ(C,x) = |C ∩ x| / |x|**: Inclusion (how much x overlaps C)
- **BSS_ρ(C,x) = |C \ x| / |x|**: Exclusion (how much C diverges from x)
- **τ**: Minimum required overlap (typically 0.7)
- **ρ**: Maximum allowed divergence (typically 0.3)

**This is the MECHANICS of contextual selection!**

In [3]:
# Configure thresholds
config = HRTConfig(
    p_bits=8,
    h_bits=16,
    tau=0.7,      # Require 70% overlap
    rho=0.3,      # Allow at most 30% divergence
    epsilon=0.1
)

print(f"Concrete Mechanics:")
print(f"  τ = {config.tau} (inclusion threshold)")
print(f"  ρ = {config.rho} (exclusion threshold)")
print(f"  Constraint: 0 ≤ ρ < τ ≤ 1 ✓")
print(f"\nSelection Rule:")
print(f"  S_C(x) = 1 ⟺ BSS_τ(C,x) ≥ {config.tau} AND BSS_ρ(C,x) ≤ {config.rho}")

Concrete Mechanics:
  τ = 0.7 (inclusion threshold)
  ρ = 0.3 (exclusion threshold)
  Constraint: 0 ≤ ρ < τ ≤ 1 ✓

Selection Rule:
  S_C(x) = 1 ⟺ BSS_τ(C,x) ≥ 0.7 AND BSS_ρ(C,x) ≤ 0.3


## Applying the Mechanics

Let's test various candidates against the quantum context.

In [4]:
# Create test candidates
candidates = [
    (['quantum', 'decoherence', 'collapse', 'measurement'], 'Related'),
    (['classical', 'deterministic', 'macroscopic', 'predictable'], 'Unrelated'),
    (['wave', 'particle', 'duality', 'quantum'], 'Partial'),
    (['superposition', 'entanglement', 'nonlocality', 'quantum'], 'Strong')
]

print(f"Testing candidates against quantum context:\n")

for idx, (tokens, label) in enumerate(candidates, start=1):
    candidate_hllset = kernel.absorb(tokens)
    candidate = BasicHLLSet(index=idx, is_row=True, hllset=candidate_hllset, config=config)
    
    # Compute BSS
    bss_tau = context.bss_tau(candidate)
    bss_rho = context.bss_rho(candidate)
    
    # Apply selection rule
    selected = context.has_morphism_to(candidate)
    
    status = "✓ SELECTED" if selected else "✗ FILTERED"
    
    print(f"{label}: {tokens}")
    print(f"  BSS_τ = {bss_tau:.2f} {'≥' if bss_tau >= config.tau else '<'} {config.tau}")
    print(f"  BSS_ρ = {bss_rho:.2f} {'≤' if bss_rho <= config.rho else '>'} {config.rho}")
    print(f"  {status}\n")

Testing candidates against quantum context:

Related: ['quantum', 'decoherence', 'collapse', 'measurement']
  BSS_τ = 0.00 < 0.7
  BSS_ρ = 1.25 > 0.3
  ✗ FILTERED

Unrelated: ['classical', 'deterministic', 'macroscopic', 'predictable']
  BSS_τ = 0.00 < 0.7
  BSS_ρ = 1.25 > 0.3
  ✗ FILTERED

Partial: ['wave', 'particle', 'duality', 'quantum']
  BSS_τ = 0.00 < 0.7
  BSS_ρ = 1.25 > 0.3
  ✗ FILTERED

Strong: ['superposition', 'entanglement', 'nonlocality', 'quantum']
  BSS_τ = 0.00 < 0.7
  BSS_ρ = 1.25 > 0.3
  ✗ FILTERED



## Priority Weighting: Extended Mechanics

**Beyond binary selection**, we can rank candidates by priority:

```text
Priority(C → x) = BSS_τ(C,x) - λ·BSS_ρ(C,x)
```

where λ ∈ [0,1] balances inclusion vs exclusion.

**Interpretation:**
- High BSS_τ → High priority (strong connection)
- High BSS_ρ → Lower priority (penalize divergence)
- **This extends selection to graded ranking**

In [5]:
# Compute priorities for all candidates
lambda_weight = 0.5

priorities = []
for idx, (tokens, label) in enumerate(candidates, start=1):
    candidate_hllset = kernel.absorb(tokens)
    candidate = BasicHLLSet(index=idx, is_row=True, hllset=candidate_hllset, config=config)
    bss_t = context.bss_tau(candidate)
    bss_r = context.bss_rho(candidate)
    priority = bss_t - lambda_weight * bss_r
    selected = context.has_morphism_to(candidate)
    priorities.append((label, bss_t, bss_r, priority, selected))

# Sort by priority
priorities.sort(key=lambda x: x[3], reverse=True)

print(f"Priority Ranking (λ={lambda_weight}):\n")
print(f"{'Candidate':<15} {'BSS_τ':<8} {'BSS_ρ':<8} {'Priority':<10} {'Selected'}")
print("-" * 60)
for label, bss_t, bss_r, pri, sel in priorities:
    status = "✓" if sel else "✗"
    print(f"{label:<15} {bss_t:<8.2f} {bss_r:<8.2f} {pri:<10.2f} {status}")

Priority Ranking (λ=0.5):

Candidate       BSS_τ    BSS_ρ    Priority   Selected
------------------------------------------------------------
Related         0.00     1.25     -0.63      ✗
Unrelated       0.00     1.25     -0.63      ✗
Partial         0.00     1.25     -0.63      ✗
Strong          0.00     1.25     -0.63      ✗


## Layer 3: Structural Result (W Lattice)

**The mechanics produce a filtered lattice:**

```text
W = {(u, v, w) : u, v ∈ H, BSS_τ(u,v) ≥ τ, BSS_ρ(u,v) ≤ ρ, w = BSS_τ(u,v)}
```

**Key Insight:** W ⊂ H×H (proper subset)

**Therefore:**
- **AM (Adjacency Matrix)**: All co-occurrences (|H| × |H|)
- **W (Weighted Lattice)**: Filtered connections (|W| < |H|²)
- **AM ≠ W**: Structure diverges due to τ/ρ filtering

In [6]:
# Create multiple contexts to demonstrate W
contexts_data = [
    (['quantum', 'entanglement', 'superposition'], 'Quantum'),
    (['classical', 'deterministic', 'macroscopic'], 'Classical'),
    (['measurement', 'observation', 'collapse'], 'Measurement'),
    (['wave', 'particle', 'duality'], 'WaveParticle')
]

contexts = [BasicHLLSet(index=idx, is_row=True, hllset=kernel.absorb(tokens), config=config)
            for idx, (tokens, label) in enumerate(contexts_data)]

print(f"Created {len(contexts)} contexts:\n")

# Build adjacency matrix (all pairs)
n = len(contexts)
print(f"Adjacency Matrix AM: {n}×{n} = {n*n} potential edges")
print(f"Upper triangle (excluding diagonal): {n*(n-1)//2} edges\n")

# Build W lattice (filtered pairs)
w_edges = []
for i, ctx_i in enumerate(contexts):
    for j, ctx_j in enumerate(contexts):
        if i < j:  # Upper triangle only
            if ctx_i.has_morphism_to(ctx_j):
                w = ctx_i.bss_tau(ctx_j)
                w_edges.append((i, j, w, contexts_data[i][1], contexts_data[j][1]))

print(f"W Lattice (filtered by τ/ρ): {len(w_edges)} edges\n")
print(f"Edges in W:")
for i, j, w, label_i, label_j in w_edges:
    print(f"  ({i},{j}): {label_i} ↔ {label_j}, w={w:.2f}")

print(f"\n✓ W ⊂ AM: {len(w_edges)} < {n*(n-1)//2} (proper subset)")
print(f"✓ AM ≠ W: Structure diverges due to contextual selection")

Created 4 contexts:

Adjacency Matrix AM: 4×4 = 16 potential edges
Upper triangle (excluding diagonal): 6 edges

W Lattice (filtered by τ/ρ): 0 edges

Edges in W:

✓ W ⊂ AM: 0 < 6 (proper subset)
✓ AM ≠ W: Structure diverges due to contextual selection


## Using ContextualSelection Class

The `ContextualSelection` class provides a high-level API for selection operations.

In [7]:
# Create test candidates
test_candidates = [
    BasicHLLSet(index=idx, is_row=True, hllset=kernel.absorb(tokens), config=config)
    for idx, (tokens, label) in enumerate(candidates, start=1)
]

# Create ContextualSelection operator
selector = ContextualSelection.from_context(
    context=context,
    candidates=test_candidates
)

print(f"ContextualSelection operator:")
print(f"  Context: {context.name}")
print(f"  τ = {selector.tau_threshold}, ρ = {selector.rho_threshold}")
print(f"  Selection power: {selector.selection_power:.2f}\n")

print(f"Applying S_C operator to candidates:\n")
for idx, cand in enumerate(test_candidates):
    # Check if candidate was selected
    is_selected = cand.hllset.name in selector.selected_hashes
    s_c = 1 if is_selected else 0
    label = candidates[idx][1]
    print(f"  S_C({label}) = {s_c}")

ContextualSelection operator:
  Context: r_0
  τ = 0.7, ρ = 0.3
  Selection power: 0.00

Applying S_C operator to candidates:

  S_C(Related) = 0
  S_C(Unrelated) = 0
  S_C(Partial) = 0
  S_C(Strong) = 0


## Complete Integration: The Full Picture

```text
┌─────────────────────────────────────────┐
│ Layer 1: Abstract Principle             │
│ "Context actively selects elements"     │
│                                         │
│ S_C: H → {0,1}                          │
│ S_C(x) = 1 ⟺ x selected by C           │
└─────────────────────────────────────────┘
              ↓ IMPLEMENTED VIA
┌─────────────────────────────────────────┐
│ Layer 2: Concrete Mechanics             │
│ τ/ρ thresholds, BSS(τ,ρ)                │
│                                         │
│ BSS_τ(C,x) = |C∩x|/|x| ≥ τ              │
│ BSS_ρ(C,x) = |C\x|/|x| ≤ ρ              │
│ Priority = BSS_τ - λ·BSS_ρ              │
└─────────────────────────────────────────┘
              ↓ PRODUCES
┌─────────────────────────────────────────┐
│ Layer 3: Structural Result              │
│ W ⊂ H×H filtered lattice                │
│                                         │
│ W = {(u,v,w): satisfies τ/ρ}            │
│ AM ≠ W (divergence)                     │
│ Priority-weighted paths                 │
└─────────────────────────────────────────┘
```

In [8]:
print("Complete Contextual Selection Integration:\n")
print("1. PRINCIPLE: Context actively selects elements")
print("   → Abstract operator S_C: H → {0,1}\n")

print("2. MECHANICS: τ/ρ thresholds and BSS weights")
print(f"   → τ={config.tau} (inclusion), ρ={config.rho} (exclusion)")
print(f"   → S_C(x) = 1 ⟺ BSS_τ(C,x) ≥ τ AND BSS_ρ(C,x) ≤ ρ\n")

print("3. RESULT: W lattice and priority-weighted structure")
print(f"   → W ⊂ H×H (filtered from {n*(n-1)//2} to {len(w_edges)} edges)")
print(f"   → AM ≠ W (structure diverges)")
print(f"   → Priority = BSS_τ - λ·BSS_ρ (graded ranking)\n")

print("✓ Principle → Mechanics → Structure")
print("✓ Abstract concept realized through concrete computation")
print("✓ τ/ρ are the mechanics that implement contextual selection")

Complete Contextual Selection Integration:

1. PRINCIPLE: Context actively selects elements
   → Abstract operator S_C: H → {0,1}

2. MECHANICS: τ/ρ thresholds and BSS weights
   → τ=0.7 (inclusion), ρ=0.3 (exclusion)
   → S_C(x) = 1 ⟺ BSS_τ(C,x) ≥ τ AND BSS_ρ(C,x) ≤ ρ

3. RESULT: W lattice and priority-weighted structure
   → W ⊂ H×H (filtered from 6 to 0 edges)
   → AM ≠ W (structure diverges)
   → Priority = BSS_τ - λ·BSS_ρ (graded ranking)

✓ Principle → Mechanics → Structure
✓ Abstract concept realized through concrete computation
✓ τ/ρ are the mechanics that implement contextual selection


## Summary

**Key Insights:**

1. **Contextual Selection** is an abstract principle: contexts actively determine what exists in them

2. **τ/ρ Mechanics** are the concrete implementation:
   - BSS_τ measures inclusion (overlap)
   - BSS_ρ measures exclusion (divergence)
   - Together they define selection criteria

3. **W Lattice** is the structural result:
   - Filtered from H×H by τ/ρ thresholds
   - W ⊂ H×H (proper subset)
   - AM ≠ W (structure diverges)

4. **Priority Weighting** extends binary selection to graded ranking:
   - Priority = BSS_τ - λ·BSS_ρ
   - Enables path selection in language generation, search, etc.

**References:**
- **DOCS/CONTEXTUAL_SELECTION_MECHANICS.md**: Visual three-layer diagram
- **DOCS/HRT_LATTICE_THEORY.md**: Complete lattice theory
- **DOCS/PRIORITY_WEIGHTING.md**: API and patterns